In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/df.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Cross validation
rmse_list = []
for train_index, test_index in tscv.split(df):
    train, test = df.iloc[train_index, :], df.iloc[test_index, :]
    model = SARIMAX(train['orders'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
    model_fit = model.fit(disp=0)
    predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1)
    rmse = sqrt(mean_squared_error(test['orders'], predictions))
    rmse_list.append(rmse)

# Print cross-validation results
print('RMSE for each fold:', rmse_list)
print('Average RMSE:', np.mean(rmse_list))

# Fit model to entire data and make predictions for next 3 days (72 hours = 3*24)
model = SARIMAX(df['orders'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
model_fit = model.fit(disp=0)
forecast = model_fit.forecast(steps=72)

# Create and save forecast dataframe
forecast_df = pd.DataFrame({'date': pd.date_range(start=df.index.max(), periods=73, freq='H', closed='right'),
                            'hour': [d.hour for d in pd.date_range(start=df.index.max(), periods=73, freq='H', closed='right')],
                            'orders': forecast})
forecast_df.to_csv('forecast.csv', index=False)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/

RMSE for each fold: [9.041572313799197, 8.641118030821522, 7.578388003779294, 7.436508541658538, 7.465650287874271]
Average RMSE: 8.032647435586565


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
<ipython-input-2-9ca13759df26>:36: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  forecast_df = pd.DataFrame({'date': pd.date_range(start=df.index.max(), periods=73, freq='H', closed='right'),
<ipython-input-2-9ca13759df26>:37: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  'hour': [d.hour for d in pd.date_range(start=df.index.max(), periods=73, freq='H', closed='right')],
